<pre>
🔒 GIL (Global Interpreter Lock) in Python:

The Global Interpreter Lock (GIL) is a mutex that prevents multiple native threads from executing Python bytecodes at once.
This lock exists in CPython (the default Python implementation) to simplify memory management.
particularly around reference counting, which isn't thread-safe by default.

Workarounds for GIL limitations:

1. Use multiprocessing module:
    Bypasses the GIL by using separate processes instead of threads.
    Each process has its own Python interpreter and memory space.
    Best for CPU-bound tasks.

2. Use I/O-bound multi-threading with threading module:
    Still effective if you're waiting on disk, network, or other I/O.

<pre>
Question:
Explain the difference between shallow copy and deep copy in Python. Provide examples to illustrate when you'd use one over the other.

Soln:
A shallow copy creates a new object, but it doesn't recursively copy the objects contained inside the original object. 
Instead, it copies the references to those inner objects.

A deep copy, on the other hand, recursively copies every object, including all nested objects. This means that the new object and all its nested objects are completely independent of the original object 
</pre>



Question:
<pre>

You have the following function:

def func(a=[]):
    a.append(1)
    return a

What will be the result of calling func() three times? Why does this happen, and how would you fix it?
<pre>

In [ ]:
def func(a=[]):   # default arg is mutable!
    a.append(1)
    return a

print(func())
print(func())
print(func())  


[1]
[1, 1]
[1, 1, 1]


<pre>
In Python, default arguments are evaluated only once at function definition time, not each time the function is called.
So the list a=[] is created only once, and that same list is reused across all calls to func() when no argument is passed.
Hence, all calls share the same list object in memory, and append(1) keeps adding to it.
fix: def func(a=None):
</pre>

### Question:
<pre>
Consider the following code: What will be printed? Explain why.

x = 10

def outer():
    x = 20
    def inner():
        nonlocal x
        x = 30
    inner()
    print(x)

outer()
print(x)
</pre>


In [ ]:
x = 10
def outer():
    x = 20   # 🔁 Enclosing (nonlocal for inner, but local to outer)
    def inner():
        nonlocal x  # ⬅️ This x refers to outer()'s x (20)
        x = 30
    inner()
    print(x)

outer()
print(x)


30
10


<pre>
Rule of nonlocal:
    It only works inside a nested function.
    It modifies variables defined in the enclosing function scope, not global scope.
    If it can’t find such a variable in the enclosing (but non-global) scope, it’ll raise an error.
</pre>

<pre>
Question:
a = [1, 2, 3]
b = a
a += [4]

print(a)  # ?
print(b)  # ?

And how would it change if we did a = a + [4] instead?
</pre>

In [ ]:
a = [1, 2, 3]
b = a          # means both a and b point to the same list in memory.
a += [4]       # in-place modification using list.__iadd__(), so the list object itself is modified.

print(a)  # ?
print(b)  # ?


[1, 2, 3, 4]
[1, 2, 3, 4]


In [9]:
a = [1, 2, 3]
b = a            # means both a and b point to the same list in memory.
a = a +[4]       # rebinds a to a new object
print(a)  # ?
print(b)  # ?

[1, 2, 3, 4]
[1, 2, 3]



### Question:
<pre>
Explain what the following does — and predict the output:
def make_counter():
    count = 0
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

c1 = make_counter()
c2 = make_counter()

print(c1(), c1(), c2(), c1(), c2())
</pre>


In [10]:
def make_counter():
    count = 0
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

c1 = make_counter()
c2 = make_counter()

print(c1(), c1(), c2(), c1(), c2())

1 2 1 3 2


<pre>
Explanation:
    make_counter() returns a new counter() function each time — and each has its own enclosed count.
    nonlocal count allows counter() to modify that enclosed count from the outer function.

</pre>

### Note:
Inner function that wraps the original function → call it wrapper

Function that receives the original function → call it decorator

Function that receives arguments and returns a decorator → call it decorator factory

### Question:
<pre>
What will this print and why? And bonus: how would you preserve function metadata?

def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@log_calls
def greet(name):
    """Greets a person."""
    print(f"Hello, {name}!")

print(greet.__name__)
print(greet.__doc__)
greet("Alice")

</pre>

In [11]:
def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@log_calls
def greet(name):
    """Greets a person."""
    print(f"Hello, {name}!")

print(greet.__name__)
print(greet.__doc__)
greet("Alice")

wrapper
None
Calling greet
Hello, Alice!


<pre>
Why this happens:

When you use a decorator like this:
@log_calls
def greet(name):
    ...

It’s equivalent to:

greet = log_calls(greet)

So now greet is the wrapper function returned by log_calls. That wrapper has:

    no docstring

    __name__ == 'wrapper'

Fix: Use the functools.wraps decorator from the functools module
@wraps updates the wrapper function to carry over the original function’s metadata
 — like __name__, __doc__, and even __annotations__ — which is crucial for introspection and tooling."
</pre>

In [12]:
from functools import wraps

def log_calls(func):
    @wraps(func)  # THIS IS THE FIX
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

